In [2]:
import numpy as np
import maxflow
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import cv2

#### Calculate $\beta$
 - Beta is the smoothness term for the boundary between bg and fg
 - When beta is 1, the boundary is perfectly smooth
 - beta > 0 is preferred to relax constraints on hard boundary b/w pixels
 
 $$
     \beta = \frac{1}{(2*\text{E}(\sqrt{||pixel_{i} - pixel{j}||})}
 $$
 
 Here, $\text{E}$ is the average over all pairs of neighbouring pixels in the image. which are $4wh -3h -3w +2$ in number.

In [3]:
class GrabCut:
    def __init__(self, img, gamma, n_components, max_iter, bgRect, mask, n_iters):
        #  x_start, x_end, y_start, y_end is what we should get after unpacking bgRect
        self.img = img.astype(np.float32)
        self.height, self.width =  list(img.shape)[:2]
        self.k = n_components
        self.gamma = gamma
        self.n_iters = n_iters
        self.bgRect = bgRect
        self.mask = mask
        self.max_iters = max_iter

        # define flags
        self.BG = 0	# confirmed bg
        self.FG = 1	# confirmed fg
        self.PR_BG = 2	# probable bg
        self.PR_FG = 3	# probable fg    